In [9]:
%pip install -q matplotlib ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [10]:
%matplotlib inline
import base64
import io
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import HTML, display

display(
    HTML(
        """
        <style>
        .function-plot img {
            width: 100%;
            height: auto;
        }
        </style>
        """
    )
)

functions = {
    "sin(x) * cos(y)": lambda x, y: np.sin(x) * np.cos(y),
    "saddle (x * y)": lambda x, y: x * y,
    "ripple sin(r^2)/(r^2+1)": lambda x, y: np.sin(x**2 + y**2) / (x**2 + y**2 + 1e-3),
    "peaks": lambda x, y: (
        3 * (1 - x) ** 2 * np.exp(-(x**2) - (y + 1) ** 2)
        - 10 * (x / 5 - x**3 - y**5) * np.exp(-x**2 - y**2)
        - 1 / 3 * np.exp(-(x + 1) ** 2 - y**2)
    ),
}

def plot_field(func_name, x_range, y_range, resolution, cmap_name):
    xmin, xmax = x_range
    ymin, ymax = y_range
    n = resolution

    x = np.linspace(xmin, xmax, n)
    y = np.linspace(ymin, ymax, n)
    X, Y = np.meshgrid(x, y)
    Z = functions[func_name](X, Y)

    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)
    im = ax.imshow(
        Z,
        extent=[xmin, xmax, ymin, ymax],
        origin="lower",
        cmap=cmap_name,
        aspect="auto",
    )
    ax.contour(X, Y, Z, levels=10, colors="white", linewidths=0.6, alpha=0.7)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(f"f(x, y) = {func_name}")
    fig.colorbar(im, ax=ax, label="f(x, y)")

    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=150)
    plt.close(fig)
    buf.seek(0)
    encoded = base64.b64encode(buf.read()).decode("ascii")
    display(
        HTML(
            f'<div class="function-plot"><img src="data:image/png;base64,{encoded}" alt="Plot of {func_name}" /></div>'
        )
    )


style = {"description_width": "110px"}
x_slider = widgets.FloatRangeSlider(
    value=(-3.0, 3.0), min=-10.0, max=10.0, step=0.1, description="x range", style=style
)
y_slider = widgets.FloatRangeSlider(
    value=(-3.0, 3.0), min=-10.0, max=10.0, step=0.1, description="y range", style=style
)
resolution_slider = widgets.IntSlider(
    value=150, min=40, max=400, step=10, description="grid size", style=style
)
cmap_dropdown = widgets.Dropdown(
    options=["viridis", "plasma", "inferno", "magma", "cividis", "turbo"],
    value="turbo",
    description="colormap",
    style=style,
)

widgets.interact(
    plot_field,
    func_name=widgets.Dropdown(options=list(functions.keys()), value="peaks", description="function", style=style),
    x_range=x_slider,
    y_range=y_slider,
    resolution=resolution_slider,
    cmap_name=cmap_dropdown,
);


interactive(children=(Dropdown(description='function', index=3, options=('sin(x) * cos(y)', 'saddle (x * y)', …